Derek Lamb
DSC 630
Term Project

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import missingno as msno

In [134]:
# Example path for EIA data 
eia_data_paths = [
    '/Users/dereklamb/downloads/MER_T01_02.csv',
    '/Users/dereklamb/downloads/MER_T10_02B.csv',
    '/Users/dereklamb/downloads/MER_T10_02A.csv',
    '/Users/dereklamb/downloads/MER_T10_01.csv'
]

# Inspect the 'YYYYMM' column to identify issues
print(eia_data['YYYYMM'].unique())  # Check for unexpected values

# Step 1: Clean the 'YYYYMM' column
# Remove non-numeric rows or malformed data
eia_data = eia_data[eia_data['YYYYMM'].astype(str).str.isnumeric()]

# Step 2: Ensure all values have the correct length (6 characters for YYYYMM)
eia_data = eia_data[eia_data['YYYYMM'].astype(str).str.len() == 6]

# Step 3: Convert 'YYYYMM' to datetime
eia_data['Date'] = pd.to_datetime(eia_data['YYYYMM'].astype(str), format='%Y%m')


# Check the first few rows to ensure the conversion worked
print(eia_data.head())


[194913 195013 195113 195213 195313 195413 195513 195613 195713 195813
 195913 196013 196113 196213 196313 196413 196513 196613 196713 196813
 196913 197013 197113 197213 197301 197302 197303 197304 197305 197306
 197307 197308 197309 197310 197311 197312 197313 197401 197402 197403
 197404 197405 197406 197407 197408 197409 197410 197411 197412 197413
 197501 197502 197503 197504 197505 197506 197507 197508 197509 197510
 197511 197512 197513 197601 197602 197603 197604 197605 197606 197607
 197608 197609 197610 197611 197612 197613 197701 197702 197703 197704
 197705 197706 197707 197708 197709 197710 197711 197712 197713 197801
 197802 197803 197804 197805 197806 197807 197808 197809 197810 197811
 197812 197813 197901 197902 197903 197904 197905 197906 197907 197908
 197909 197910 197911 197912 197913 198001 198002 198003 198004 198005
 198006 198007 198008 198009 198010 198011 198012 198013 198101 198102
 198103 198104 198105 198106 198107 198108 198109 198110 198111 198112
 19811

ValueError: unconverted data remains when parsing with format "%Y%m": "3", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# Load NOAA and CIMIS datasets
noaa_data_path = pd.read_csv('/Users/dereklamb/downloads/Final_Combined_Weather_Data(in).csv',encoding='ISO-8859-1')
cimis_data = pd.read_csv('/Users/dereklamb/downloads/all_conditions(in).csv')

# Quick view of the NOAA and CIMIS data
print(noaa_data.head())
print(cimis_data.head())


In [ ]:
# Check for missing values in each dataset
print(eia_data.isnull().sum())
print(noaa_data.isnull().sum())
print(cimis_data.isnull().sum())

# Visualize missing data
msno.matrix(noaa_data)
plt.show()

msno.matrix(eia_data)
plt.show()


In [ ]:
# Handle missing data
# Forward fill missing values for weather data
noaa_data.fillna(method='ffill', inplace=True)
# Drop rows with missing values in EIA data
eia_data.dropna(inplace=True)

# Example of dropping rows with excessive missing values
cimis_data = cimis_data.dropna(thresh=cimis_data.shape[1] - 2)


In [ ]:
# Descriptive statistics for all datasets
print(eia_data.describe())
print(noaa_data.describe())
print(cimis_data.describe())


In [ ]:
print(cimis_data.head(10))


# Inspect the columns of the DataFrame to ensure they match exactly
print(cimis_data.columns)



In [ ]:
# Inspect the data types
print(noaa_data.info())

# Check for unique values in columns to identify potential non-numeric data
print(noaa_data.head())

# Drop non-numeric columns before computing the correlation matrix
numeric_noaa_data = noaa_data.select_dtypes(include=[np.number])

# Select only numeric columns from CIMIS data for the correlation matrix
numeric_cimis_data = cimis_data.select_dtypes(include=[float, int])

# Compute the correlation matrix
correlation_matrix_cimis = numeric_cimis_data.corr()

# Check if the matrix is non-empty and print it
print(correlation_matrix_cimis)

# Compute the correlation matrix
correlation_matrix_cimis = numeric_cimis_data.corr()

# Check if the matrix is non-empty
print(correlation_matrix_cimis)

# Attempt to convert all columns to numeric, setting errors='coerce' to replace non-convertible values with NaN
noaa_data = noaa_data.apply(pd.to_numeric, errors='coerce')

# Now, drop rows with NaN values if necessary
noaa_data.dropna(inplace=True)

# Compute correlation matrix for NOAA weather data
correlation_matrix_noaa = noaa_data.corr()

# Heatmap of the NOAA correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_noaa, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of NOAA Weather Data')
plt.show()


In [ ]:

# Plot time series of energy production over time from EIA data
plt.figure(figsize=(12, 6))
plt.plot(eia_data['Date'], eia_data['Energy_Production'], label='Energy Production')
plt.title('Energy Production Over Time')
plt.xlabel('Date')
plt.ylabel('Energy Production (MW)')
plt.legend()
plt.show()


In [ ]:
# Scatter plot to explore the relationship between solar radiation and energy production
plt.figure(figsize=(10, 6))
plt.scatter(noaa_data['Sol Rad (Ly/day)'], eia_data['Energy_Production'])
plt.title('Solar Radiation vs. Energy Production')
plt.xlabel('Solar Radiation (Ly/day)')
plt.ylabel('Energy Production (MW)')
plt.show()


In [ ]:
# Scatter plot to explore the relationship between wind speed and wind energy production
plt.figure(figsize=(10, 6))
plt.scatter(noaa_data['Avg Wind Speed (mph)'], eia_data['Wind_Energy_Production'])
plt.title('Wind Speed vs. Wind Energy Production')
plt.xlabel('Average Wind Speed (mph)')
plt.ylabel('Wind Energy Production (MW)')
plt.show()


In [ ]:
# Select columns to scale
columns_to_scale = ['Sol Rad (Ly/day)', 'Max Air Temp (F)', 'Avg Wind Speed (mph)', 'Avg Rel Hum (%)']

# Scale the data using StandardScaler
scaler = StandardScaler()
noaa_data[columns_to_scale] = scaler.fit_transform(noaa_data[columns_to_scale])

# Preview the scaled data
print(noaa_data.head())


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define features and target
X = noaa_data[['Sol Rad (Ly/day)', 'Avg Wind Speed (mph)', 'Max Air Temp (F)', 'Avg Rel Hum (%)']]
y = eia_data['Energy_Production']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}, RMSE: {rmse}, R^2: {r2}')
